In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
from google.colab import drive 
from scipy.stats import norm
#SkLearn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
#Tensoflow
import tensorflow as tf

In [2]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [47]:
edad = np.array([16,25,39,45,49,64,70])
peso = np.array([140,149,165,170,165,159,144])
presion= np.array([125,129,127,150,161,144,132])
x = np.column_stack((edad,peso))
y = presion

In [25]:
df = pd.DataFrame(x)

In [26]:
df

,0,1
0,16,140
1,25,149
2,39,165
3,45,170
4,49,165
5,64,159
6,70,144


In [27]:
poly = PolynomialFeatures(2)
data_poly = poly.fit_transform(df)
data_poly

array([[1.0000e+00, 1.6000e+01, 1.4000e+02, 2.5600e+02, 2.2400e+03,
        1.9600e+04],
       [1.0000e+00, 2.5000e+01, 1.4900e+02, 6.2500e+02, 3.7250e+03,
        2.2201e+04],
       [1.0000e+00, 3.9000e+01, 1.6500e+02, 1.5210e+03, 6.4350e+03,
        2.7225e+04],
       [1.0000e+00, 4.5000e+01, 1.7000e+02, 2.0250e+03, 7.6500e+03,
        2.8900e+04],
       [1.0000e+00, 4.9000e+01, 1.6500e+02, 2.4010e+03, 8.0850e+03,
        2.7225e+04],
       [1.0000e+00, 6.4000e+01, 1.5900e+02, 4.0960e+03, 1.0176e+04,
        2.5281e+04],
       [1.0000e+00, 7.0000e+01, 1.4400e+02, 4.9000e+03, 1.0080e+04,
        2.0736e+04]])

In [38]:
x= data_poly
x_train = x
y_train = y

In [43]:
tf.reset_default_graph()
g = tf.Graph()
n = len(x)
with g.as_default():
  tensor_x = tf.placeholder(tf.float32,[None, 6],"tensor_x")
  tensor_y = tf.placeholder(tf.float32,[None, 1],"tensor_y")
  learning_rate = tf.placeholder("float") 

  #Parameters
  with tf.name_scope("parameters"):
    m = tf.get_variable("weights",dtype=tf.float32,  initializer=tf.zeros(7,1))
    b = tf.get_variable("b",dtype=tf.float32, initializer=tf.zeros(1))

  # Hypothesis
  with tf.name_scope("hypothesis"):
   y_pred = tf.multiply(m, tensor_x)
    
  # Mean Squared Error Cost Function
  with tf.name_scope("Error-function"):
    cost = tf.reduce_sum(tf.pow(y_pred-tensor_y, 2)) / 1/2
    #Cost to Tensorboard  
    cost_summary = tf.summary.scalar("cost", cost)

  with tf.name_scope("Gradient"):

    #Goal: Gradient Descent Optimizer
    optimizer = tf.gradients(cost ,[m,b])  
    actualizacion_m = tf.assign(m, m - learning_rate*optimizer[0] )
    actualizacion_b = tf.assign(b, b - learning_rate*optimizer[1] )
    actualizacion_parametros = tf.group(actualizacion_m,actualizacion_b)

  # Global Variables Initializer
  init = tf.global_variables_initializer()

In [44]:
def polynomial_model(lr, epochs, batch_size, x, y):
  now = datetime.now()
  total_iteraciones = int(n/batch_size)
  with tf.train.MonitoredSession() as session:
    session = tf.Session(graph = g)
    session.run(init)
    writer = tf.summary.FileWriter("exp/"+str(now)+"n="+str(epochs)+" lr="+str(lr), session.graph)
    for epoch in range(epochs): 
      for i in range(total_iteraciones): 
        muestra_inicio = i*batch_size
        muestra_fin = muestra_inicio + batch_size

        #nuevo arreglo de muestra mas pequena
        x_mb =  np.array(x[muestra_inicio:muestra_fin]).reshape([-1,6])
        y_mb = np.array(y[muestra_inicio:muestra_fin]).reshape([-1,1])
        #print(y_mb.shape, x_mb.shape)
        
        feed_dict = {tensor_x:x_mb, tensor_y:y_mb, learning_rate:lr}
        w=session.run(y_pred,feed_dict=feed_dict)
        session.run([actualizacion_parametros,m,b],feed_dict=feed_dict)

      feed_dict = {tensor_x:np.array(x).reshape(-1,6), tensor_y:np.array(y).reshape(-1,1), learning_rate:lr}
      predicciones, cs = session.run([y_pred, cost_summary] ,feed_dict=feed_dict)
      writer.add_summary(cs, epoch+1)

  pendiente, intercepto = session.run([m,b],feed_dict=feed_dict)
  return pendiente, intercepto

In [48]:
m1_m,m1_b = polynomial_model(0.01, 100, 8, x, y)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [52]:
print(m1_m, m1_b)

0.699002996596792 0.194208614510255
